In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load data
data = pd.read_csv("/content/drive/My Drive/7Train1.csv", header=None)

# Function to create big drop labels for the training dataset (threshold = 5)
def add_big_drop_label_train(bandwidth, threshold=5):
    big_drop = []
    for i in range(0, len(bandwidth)-1):
        diff = abs(bandwidth[i+1] - bandwidth[i])
        big_drop.append(1 if diff >= threshold else 0)
    return big_drop + [0]  # Add initial 0 for the first value

# Function to create big drop labels for the testing dataset (threshold = 3)
def add_big_drop_label_test(bandwidth, threshold=5):
    big_drop = []
    for i in range(1, len(bandwidth)):
        diff = abs(bandwidth[i] - bandwidth[i-1])
        big_drop.append(1 if diff >= threshold else 0)
    return [0] + big_drop  # Add initial 0 for first value

# Prepare bandwidth and big drop labels for the training set
bandwidth = data.iloc[:, 0].values  # Convert to NumPy array
len_train = math.floor(len(bandwidth) * 0.8)

for i in range(len(bandwidth)):
    if bandwidth[i] > 30:
        bandwidth[i] = 0

scaler = MinMaxScaler(feature_range=(-1, 1))
bandwidth_normalized = scaler.fit_transform(bandwidth.reshape(-1, 1))

# Add big drop labels to the training dataset
big_drop_labels_train = torch.tensor(add_big_drop_label_train(bandwidth), dtype=torch.float32).view(-1, 1)

# Combine bandwidth and big drop labels for the training dataset
input_features_train = np.column_stack((bandwidth_normalized, big_drop_labels_train))

# Convert to PyTorch tensor (input will have 2 features per timestep now)
data_tensor_train = torch.FloatTensor(input_features_train)

# Add big drop labels to the testing dataset with the modified threshold (threshold = 5)
bandwidth_test = bandwidth_normalized[len_train:]

data_tensor_test = torch.FloatTensor([0,0,0,0,0]+[x>=0.5 for x in test_predictions])
big_drop_labels_test = torch.tensor(add_big_drop_label_test(data_tensor_test), dtype=torch.float32).view(-1, 1)

# Manually define test features
# Assuming you want to input your own test features for the test set


# Convert to PyTorch tensor for the test set
# Assuming the previous part of the code where you have bandwidth and test predictions ready.

# Combine the test predictions with the original bandwidth to form a new dataset with two features
def create_test_features_with_predictions(bandwidth_test, predictions):
    # Ensure bandwidth_test and predictions have the same length
    assert len(bandwidth_test) == len(predictions), "Bandwidth and predictions lengths must match."

    # Combine bandwidth and predictions into a 2D array (two features)
    test_features = np.column_stack((bandwidth_test, predictions))

    return test_features

# Assuming `bandwidth_test` and `predictions` are already defined:
test_features_with_predictions = create_test_features_with_predictions(bandwidth_test, data_tensor_test)

# Convert to PyTorch tensor for further use
data_tensor_test_with_predictions = torch.FloatTensor(test_features_with_predictions)

# Now, you can use this combined feature set for any further processing or model predictions



# Prediction size
predict_size = 1

# Function to create in-out sequences
def create_inout_sequences_with_big_drop(input_data, big_drop_labels, window_size, predict_size):
    inout_seq = []
    L = len(input_data)
    for i in range(L - window_size - predict_size + 1):
        train_seq = input_data[i:i + window_size, :]
        train_label_bandwidth = input_data[i + window_size:i + window_size + predict_size, 0]  # Predict only bandwidth
        train_label_big_drop = big_drop_labels[i + window_size:i + window_size + predict_size]
        inout_seq.append((train_seq, (train_label_bandwidth, train_label_big_drop)))
    return inout_seq

# Parameters
window_size = 5
batch_size = 4

# Create sequences for training from the first part of the data
train_inout_seq = create_inout_sequences_with_big_drop(input_features_train[:len_train], big_drop_labels_train, window_size, predict_size)

# Custom dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx]

# Create DataLoader for training
train_dataset = TimeSeriesDataset(train_inout_seq)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# Create sequences for testing using the user-defined test features
test_inout_seq = create_inout_sequences_with_big_drop(test_features_with_predictions,big_drop_labels_test, window_size, predict_size)



class Attention(nn.Module):
    def __init__(self, hidden_layer_size, out_channel):
        super(Attention, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(out_channel, hidden_layer_size)

    def forward(self, query, key, value):
        # Ensure query, key, and value have compatible shapes
        scores = torch.bmm(self.linear(key), query.unsqueeze(2))  # Adjusting to [batch_size, hidden_size, 1]
        attn_weights = self.sigmoid(scores)
        context = torch.bmm(attn_weights.transpose(1, 2), value)
        return context, attn_weights


class LSTMWithAttention(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=128, output_size_bandwidth=1, output_size_classification=1, num_layers=2, dropout=0.2, cnn_kernel_size=10, memory_size=16, out_channel=128):
        super(LSTMWithAttention, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.out_channel = out_channel
        self.num_layers = num_layers
        self.memory_size = memory_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, dropout=dropout, batch_first=True)
        self.attention = Attention(hidden_layer_size, out_channel)
        self.cnn = nn.Linear(memory_size, out_channel)
        self.linear_bandwidth = nn.Linear(hidden_layer_size, output_size_bandwidth)
        self.linear_classification = nn.Linear(hidden_layer_size, output_size_classification)
        self.sigmoid = nn.Sigmoid()
    def forward(self, input_seq, hidden_state, past_hidden_states):
        lstm_out, hidden_state = self.lstm(input_seq, hidden_state)
        lstm_out_last = lstm_out[:, -1, :]  # Get the last time step

        # Append new hidden state to past hidden states
        past_hidden_states = torch.cat((past_hidden_states, lstm_out_last.unsqueeze(0)), dim=0)
        if past_hidden_states.size(0) > self.memory_size:
            past_hidden_states = past_hidden_states[-self.memory_size:]

        # CNN on past hidden states
        cnn_out = self.cnn(past_hidden_states.transpose(0, 1).transpose(1, 2))

        # Attention mechanism
        query = lstm_out_last  # Ensure query has same batch size as key
        context, _ = self.attention(query, cnn_out, cnn_out)
        
        # Predictions
        bandwidth_prediction = self.linear_bandwidth(context.squeeze(1))
        big_drop_prediction = self.sigmoid(self.linear_classification(context.squeeze(1)))
        return bandwidth_prediction, big_drop_prediction, hidden_state, past_hidden_states

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_layer_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_layer_size)) 


# Initialize the model, loss functions, and optimizer
model = LSTMWithAttention(input_size=2, dropout=0.2, num_layers=2)
mse_loss_function = nn.MSELoss()
bce_loss_function = nn.BCELoss()  # Using BCE for classification
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training the model
epochs = 15
train_losses = []

for epoch in range(epochs):
    epoch_train_loss = 0
    for seq, (labels_bandwidth, labels_big_drop) in train_loader:
        hidden_state = model.init_hidden(batch_size)
        past_hidden_states = torch.zeros(model.memory_size, batch_size, model.hidden_layer_size) 
        
        optimizer.zero_grad()

        seq = seq.view(batch_size, window_size, 2).float()
        labels_bandwidth = labels_bandwidth.view(batch_size, -1).float()
        labels_big_drop = labels_big_drop.view(batch_size, -1).float()

        # Forward pass
        y_pred_bandwidth, y_pred_big_drop, hidden_state, past_hidden_states = model(seq, hidden_state, past_hidden_states)
        hidden_state = (hidden_state[0].detach(), hidden_state[1].detach())
        past_hidden_states = past_hidden_states.detach()

        # Calculate losses
        bandwidth_loss = mse_loss_function(y_pred_bandwidth, labels_bandwidth)
        classification_loss = bce_loss_function(y_pred_big_drop, labels_big_drop)
        total_loss = bandwidth_loss + classification_loss

        # Backward pass
        total_loss.backward()
        optimizer.step()

        epoch_train_loss += total_loss.item()

    train_losses.append(epoch_train_loss / len(train_loader))
    if epoch % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Total Loss: {epoch_train_loss / len(train_loader):.8f}')


# Testing phase
model.eval()
hidden_state = model.init_hidden(1)
past_hidden_states = torch.zeros(model.memory_size, 1, model.hidden_layer_size)

predictions = []
big_drop_predictions = []
for seq, (labels_bandwidth, labels_big_drop) in test_inout_seq:
    seq = torch.tensor(seq, dtype=torch.float32)
    seq = seq.unsqueeze(0)
    seq = seq.view(1, window_size, -1)  # Adjust input size t
    with torch.no_grad():
        y_pred_bandwidth, y_pred_big_drop, hidden_state, past_hidden_states = model(seq, hidden_state, past_hidden_states)
        predictions.append(y_pred_bandwidth.item())
        big_drop_predictions.append(y_pred_big_drop.item())
        hidden_state = (hidden_state[0].detach(), hidden_state[1].detach())
        past_hidden_states = past_hidden_states.detach()


predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actual_values = bandwidth[len_train + window_size + predict_size - 1:]

# Calculate MAE and RMSE for all test data
# Calculate MAE and RMSE for all test data
mae = mean_absolute_error(actual_values, predictions)
rmse = np.sqrt(mean_squared_error(actual_values, predictions))
import matplotlib.pyplot as plt

# Mean Absolute Error (MAE) and RMSE Calculation
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
error_ratio_rmse = (rmse / np.mean(actual_values)) * 100
error_ratio_mae = (mae / np.mean(actual_values)) * 100
print(f'Error Ratio RMSE: {error_ratio_rmse:.4f}%')
print(f'Error Ratio MAE: {error_ratio_mae:.4f}%')

# Slice for 200 values for plotting
plot_length = 200
predictions_200 = predictions[:plot_length]
actual_values_200 = actual_values[:plot_length]

# Now we find the corresponding "big drop" labels for both actual and predicted values
big_drop_actual_200 = big_drop_labels_test[ window_size: window_size + plot_length]
big_drop_pred_200 = big_drop_predictions[:plot_length]

# Plot the actual and predicted values
plt.figure(figsize=(12, 6))
x_range = range(len_train + window_size, len_train + window_size + plot_length)

# Plot the actual bandwidth values
plt.plot(x_range, actual_values_200, label='Actual Data', color='blue', linewidth=1.5)

# Plot the predicted bandwidth values
plt.plot(x_range, predictions_200, label='Predicted Data', color='orange', linestyle='--', linewidth=1.5)

# Add blue dots for actual big drops
for i in range(plot_length):
    if big_drop_actual_200[i] == 1:
        plt.scatter(x_range[i], actual_values_200[i], color='blue', s=100, marker='o', label='Actual Big Drop' if i == 0 else "")

# Add red dots for predicted big drops
for i in range(plot_length):
    if big_drop_pred_200[i] == 1:
        plt.scatter(x_range[i], predictions_200[i], color='red', s=100, marker='o', label='Predicted Big Drop' if i == 0 else "")

# Add labels and legend
plt.legend(loc='upper right')
plt.xlabel("Index")
plt.ylabel("Bandwidth")
plt.title("Actual vs Predicted Bandwidth with Big Drops (200 Samples)")
plt.show()